In [4]:
import time
import requests
import pandas as pd

# Парсинг показателей университетов с сайта [«Приоритета 2030»](https://priority2030.ru/analytics/)

Для разных целей может понадобиться сводная таблица с текущими значениями, отражающими показатели университетов — участников программы «Приоритет 2030». Ниже представленное позволяет собрать всё в один файл.

## Список университетов

Для начал сформируем список университетов:

In [5]:
data_folder = '../data/PSAL-19'
url_base = 'https://api.socio.center/public/priority'
url_list = f"{url_base}/list"
university_list_resp = requests.get(url_list)
university_list = pd.DataFrame()

if university_list_resp.status_code == 200:
    university_list_json_data = university_list_resp.json()

    if university_list_json_data['status'] == 'success':
        university_list = pd.DataFrame(university_list_json_data['data']['participants'])
        university_list['group'] = university_list['group'].apply(lambda x: university_list_json_data['data']['group'][x])
        
        # Сохраняем полученный список университетов
        university_list.to_csv(f"{data_folder}/university_list.csv", index=False)
        university_list.to_excel(f"{data_folder}/university_list.xlsx", index=False)
else:
    raise Exception(f"{url_list} return status code {university_list_resp.status_code}")

# Выводим первые 10 записей
university_list.head(10)

,id,name,shortName,city,location,region,regionCode,founder,state,level,group
0,rmpvygmvxo,Федеральное государственное автономное образов...,МФТИ,Москва,Москва,Центральный,central,Министерство науки и высшего образования Росси...,participant,1,Участники программы (основной трек)
1,qkaxxbqxbm,Федеральное государственное бюджетное образова...,ОГУ,Оренбург,Оренбургская область,Приволжский,volga,Министерство науки и высшего образования Росси...,participant,base,Участники программы (основной трек)
2,vbclqozcgs,Федеральное государственное бюджетное образова...,ВятГУ,Киров,Кировская область,Приволжский,volga,Министерство науки и высшего образования Росси...,participant,base,Участники программы (основной трек)
3,krdeociohk,Федеральное государственное бюджетное образова...,ЮРГПУ(НПИ),Новочеркасск,Ростовская область,Южный,south,Министерство науки и высшего образования Росси...,participant,base,Участники программы (основной трек)
4,fgnzen3sxz,Федеральное государственное бюджетное образова...,ТГУ имени Г.Р. Державина,Тамбов,Тамбовская область,Центральный,central,Министерство науки и высшего образования Росси...,participant,base,Участники программы (основной трек)
5,7mzkpv35zl,Федеральное государственное бюджетное образова...,НИУ МГСУ,Москва,Москва,Центральный,central,Министерство науки и высшего образования Росси...,participant,2,Участники программы (основной трек)
6,ezg3l1cmfo,Федеральное государственное бюджетное образова...,ТулГУ,Тула,Тульская область,Центральный,central,Министерство науки и высшего образования Росси...,participant,base,Участники программы (основной трек)
7,tj72fyldee,Федеральное государственное бюджетное образова...,СПбГУПТД,Санкт-Петербург,Санкт-Петербург,Северо-Западный,north,Министерство науки и высшего образования Росси...,participant,base,Участники программы (основной трек)
8,heo0www7bu,Федеральное государственное бюджетное образова...,ФГБОУ ВО Вавиловский университет,Саратов,Саратовская область,Приволжский,volga,Министерство сельского хозяйства Российской Фе...,participant,base,Участники программы (основной трек)
9,vt81dqczsu,Федеральное государственное бюджетное образова...,"ФГБОУ ВО ""ПИМУ"" Минздрава России",Нижний Новгород,Нижегородская область,Приволжский,volga,Министерство здравоохранения Российской Федерации,participant,2,Участники программы (основной трек)


## Собираем список показателей

Чтобы не мучить сильно сайт, по каждому университету данный собираются с задержкой в 0,5 секунды, соответственно собирается всё около 1 минуты.

In [7]:
university_columns = ['ID университета', 'Университет', 'Город', 'Регион', 'Ведомство']
university_kpis = pd.DataFrame(columns=university_columns)
university_kpis_components = pd.DataFrame(columns=university_columns)   

def get_df_by_indicator(d: dict) -> pd.DataFrame:
    result = pd.DataFrame(index=[str(x) for x in d['labels']], columns=['Индикатор'])
    
    for el in d['data']:
        chunk = pd.DataFrame({'year': [str(x) for x in el['data'].keys()], f"{el['description']}": el['data'].values()}).set_index('year')
        result = result.merge(chunk, how='left', left_index=True, right_index=True)
        
    result['Индикатор'] = d['indicator']
    
    return result.reset_index().rename(columns={'index': 'Год'})


def get_df_by_group(d: dict) -> pd.DataFrame:
    result = pd.DataFrame(columns=['Группа'])
    
    for indicator in d['elements']:
        result = pd.concat([result, get_df_by_indicator(indicator)], ignore_index=True)
        
    result['Группа'] = d['name']
    
    return result


for idx, r in university_list.iterrows():
    university_id, university_short_name = r['id'], r['shortName']
    url_kpi = f"{url_base}/{university_id}/indicators"
    resp = requests.get(url_kpi)
    
    if resp.status_code == 200:
        json_data = resp.json()

        if json_data['status'] == 'success':
            for group in json_data['data']:
                df_by_group = get_df_by_group(group)
                df_by_group[['ID университета', 'Университет', 'Город', 'Регион', 'Ведомство']] = r[['id', 'shortName', 'city', 'location', 'founder']]
                university_kpis = pd.concat([university_kpis, df_by_group])
            
    else:
        raise Exception(f"{url_kpi} return status code {resp.status_code}")
        
    time.sleep(0.5)

# Сохраняем полученный список университетов
university_kpis.to_csv(f"{data_folder}/university_kpis.csv", index=False)
university_kpis.to_excel(f"{data_folder}/university_kpis.xlsx", index=False)

,ID университета,Университет,Город,Регион,Ведомство,Группа,Год,Индикатор,Отражение факта,План,Факт
0,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2021,ПРГ2,56.0,37.0,NaN
1,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2022,ПРГ2,67.0,44.0,NaN
2,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2023,ПРГ2,62.0,54.0,NaN
3,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2024,ПРГ2,NaN,60.0,NaN
4,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2025,ПРГ2,NaN,60.0,NaN
5,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2026,ПРГ2,NaN,60.0,NaN
6,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2027,ПРГ2,NaN,59.0,NaN
7,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2028,ПРГ2,NaN,59.0,NaN
8,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2029,ПРГ2,NaN,56.0,NaN
9,rmpvygmvxo,МФТИ,Москва,Москва,Министерство науки и высшего образования Росси...,"Показатели, необходимые для достижения результ...",2030,ПРГ2,NaN,55.0,NaN


Вроде бы всё 👍